### **Paso 2.4 - Ingesta del archivo "drivers.json"**

Nos permite crear e indicar parámetros en tiempo de ejecución

<center><img src="https://i.postimg.cc/brx8G9YK/db71.png"></center>

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
v_data_source

Out[3]: 'testing'

In [ ]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
v_file_date

Out[5]: '2023-06-11'

In [ ]:
%run "../includes/configuration"

In [ ]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo JSON

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [ ]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
  
])

In [ ]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)  
])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
drivers_df = spark.read \
.schema(drivers_schema) \
.json(f"{raw_folder_path}/drivers.json")
#.json(f"{raw_folder_path}/{v_file_date}/drivers.json")

In [ ]:
drivers_df.show(truncate=False)

+--------+----------+------+----+---------------------+----------+-----------+-----------------------------------------------------+
|driverId|driverRef |number|code|name                 |dob       |nationality|url                                                  |
+--------+----------+------+----+---------------------+----------+-----------+-----------------------------------------------------+
|1       |hamilton  |44    |HAM |{Lewis, Hamilton}    |1985-01-07|British    |http://en.wikipedia.org/wiki/Lewis_Hamilton          |
|2       |heidfeld  |null  |HEI |{Nick, Heidfeld}     |1977-05-10|German     |http://en.wikipedia.org/wiki/Nick_Heidfeld           |
|3       |rosberg   |6     |ROS |{Nico, Rosberg}      |1985-06-27|German     |http://en.wikipedia.org/wiki/Nico_Rosberg            |
|4       |alonso    |14    |ALO |{Fernando, Alonso}   |1981-07-29|Spanish    |http://en.wikipedia.org/wiki/Fernando_Alonso         |
|5       |kovalainen|null  |KOV |{Heikki, Kovalainen} |1981-10-19|Fin

In [ ]:
drivers_df.printSchema()

root
 |-- driverId: integer (nullable = true)
 |-- driverRef: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- code: string (nullable = true)
 |-- name: struct (nullable = true)
 |    |-- forename: string (nullable = true)
 |    |-- surname: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)



#### Paso 2 - Renombrar columnas y añadir nuevas columnas
1. **driverId** renombrado a **driver_id**
2. **driverRef** renombrado a **driver_ref**
3. Se añade la columna **ingestion_date**
4. Se añade la columna **name** con la concatenación de **forename** y **surname**.

In [ ]:
from pyspark.sql.functions import col, concat, lit

In [ ]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                    .withColumn("data_source", lit(v_data_source)) \
                                    .withColumn("file_date", lit(v_file_date))

In [ ]:
drivers_with_columns_df.show(truncate=False)

+---------+----------+------+----+------------------+----------+-----------+-----------------------------------------------------+-----------+----------+
|driver_id|driver_ref|number|code|name              |dob       |nationality|url                                                  |data_source|file_date |
+---------+----------+------+----+------------------+----------+-----------+-----------------------------------------------------+-----------+----------+
|1        |hamilton  |44    |HAM |Lewis Hamilton    |1985-01-07|British    |http://en.wikipedia.org/wiki/Lewis_Hamilton          |testing    |2023-06-11|
|2        |heidfeld  |null  |HEI |Nick Heidfeld     |1977-05-10|German     |http://en.wikipedia.org/wiki/Nick_Heidfeld           |testing    |2023-06-11|
|3        |rosberg   |6     |ROS |Nico Rosberg      |1985-06-27|German     |http://en.wikipedia.org/wiki/Nico_Rosberg            |testing    |2023-06-11|
|4        |alonso    |14    |ALO |Fernando Alonso   |1981-07-29|Spanish    |

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
drivers_with_ingestion_date_df = add_ingestion_date(drivers_with_columns_df)

In [ ]:
drivers_with_ingestion_date_df.show(truncate=False)

+---------+----------+------+----+------------------+----------+-----------+-----------------------------------------------------+-----------+----------+-----------------------+
|driver_id|driver_ref|number|code|name              |dob       |nationality|url                                                  |data_source|file_date |ingestion_date         |
+---------+----------+------+----+------------------+----------+-----------+-----------------------------------------------------+-----------+----------+-----------------------+
|1        |hamilton  |44    |HAM |Lewis Hamilton    |1985-01-07|British    |http://en.wikipedia.org/wiki/Lewis_Hamilton          |testing    |2023-06-11|2023-06-11 14:21:40.131|
|2        |heidfeld  |null  |HEI |Nick Heidfeld     |1977-05-10|German     |http://en.wikipedia.org/wiki/Nick_Heidfeld           |testing    |2023-06-11|2023-06-11 14:21:40.131|
|3        |rosberg   |6     |ROS |Nico Rosberg      |1985-06-27|German     |http://en.wikipedia.org/wiki/Nico_

#### Paso 3 - Eliminar las columnas no deseadas

In [ ]:
drivers_final_df = drivers_with_ingestion_date_df.drop(col("url"))

In [ ]:
drivers_final_df.show(truncate=False)

+---------+----------+------+----+------------------+----------+-----------+-----------+----------+-----------------------+
|driver_id|driver_ref|number|code|name              |dob       |nationality|data_source|file_date |ingestion_date         |
+---------+----------+------+----+------------------+----------+-----------+-----------+----------+-----------------------+
|1        |hamilton  |44    |HAM |Lewis Hamilton    |1985-01-07|British    |testing    |2023-06-11|2023-06-11 14:25:00.971|
|2        |heidfeld  |null  |HEI |Nick Heidfeld     |1977-05-10|German     |testing    |2023-06-11|2023-06-11 14:25:00.971|
|3        |rosberg   |6     |ROS |Nico Rosberg      |1985-06-27|German     |testing    |2023-06-11|2023-06-11 14:25:00.971|
|4        |alonso    |14    |ALO |Fernando Alonso   |1981-07-29|Spanish    |testing    |2023-06-11|2023-06-11 14:25:00.971|
|5        |kovalainen|null  |KOV |Heikki Kovalainen |1981-10-19|Finnish    |testing    |2023-06-11|2023-06-11 14:25:00.971|
|6      

In [ ]:
drivers_final_df.printSchema()

root
 |-- driver_id: integer (nullable = true)
 |-- driver_ref: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- nationality: string (nullable = true)
 |-- data_source: string (nullable = false)
 |-- file_date: string (nullable = false)
 |-- ingestion_date: timestamp (nullable = false)



#### Paso 4 - Escribir datos en el contenedor **processed** del ADLS como **parquet**

In [ ]:
# Escribimos el archivo con formato PARQUET en el contenedor "processed" y directorio "constructors"
# El parámetro "processed_folder_path" se encuentra en el notebook "configuration"
drivers_final_df.write.mode('overwrite').parquet(f"{processed_folder_path}/drivers")

In [ ]:
# Visualizamos los objetos que se encuentran en el directorio "constructors"
%fs
ls /mnt/formula1dl/processed/drivers